### Importing the necessary dependencies:

In [102]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [103]:
import pandas as pd
import numpy as np

In [104]:
from sklearn.utils import resample
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

### Importing the data files:

In [265]:
#statistical analysis on features:
df=pd.read_csv('/home/vboxuser/VoiceFiles/ImageFolder/ImageFolderopensmile_daic_dev_stat.csv')
#train and test data:
df_train=pd.read_csv('/home/vboxuser/VoiceFiles/opensmile_daic_train.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/opensmile_daic_test.csv')

In [235]:
len(df_train['file'][56])

97

In [249]:
for i in range (0,len(df_train_upsampled)):
    df_train_upsampled['file'][i]=df_train_upsampled['file'][i][80:83]

/tmp/ipykernel_16683/428126285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_upsampled['file'][i]=df_train_upsampled['file'][i][80:83]


In [251]:
sp=df_train_upsampled['file']

In [269]:
for i in range (0,len(df_test)):
    df_test['file'][i]=df_test['file'][i][79:82]

/tmp/ipykernel_16683/2951792349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['file'][i]=df_test['file'][i][79:82]


In [268]:
df_test['file'][1][79:82]

'602'

In [270]:
spt=df_test['file']

In [271]:
spt

0     600
1     602
2     604
3     605
4     606
5     607
6     609
7     615
8     618
9     619
10    620
11    622
12    623
13    624
14    625
15    626
16    629
17    631
18    634
19    635
20    636
21    637
22    638
23    640
24    649
25    650
26    651
27    652
28    655
29    656
30    658
31    659
32    661
33    663
34    664
35    666
36    669
37    676
38    679
39    682
40    683
41    688
42    689
43    691
44    693
45    696
46    699
47    705
48    708
49    709
50    710
51    712
52    715
53    716
54    717
55    718
Name: file, dtype: object

## Building a model with feature selection:

### Preprocessing the data:

#### 1. Extract the features that are a good discriminant:

In [106]:
#from the {ImageFolder..} file obtained earlier from feature_analysis we take those values that have p_val<0.05 and effect_size>|0.4|
req_features=df[(df['p_HC_DP']<0.05) & (abs(df['p_HC_DP.1'])>0.4)]

#### 2. Obtain the selected good features as a list:

In [107]:
feature_list=req_features['Unnamed: 0'].tolist()
print(feature_list)

['mfcc4_sma3_amean', 'F1amplitudeLogRelF0_sma3nz_amean', 'F1amplitudeLogRelF0_sma3nz_stddevNorm', 'F2amplitudeLogRelF0_sma3nz_amean', 'F3amplitudeLogRelF0_sma3nz_amean', 'alphaRatioV_sma3nz_amean', 'alphaRatioV_sma3nz_stddevNorm', 'hammarbergIndexV_sma3nz_amean', 'slopeUV0-500_sma3nz_amean', 'MeanUnvoicedSegmentLength', 'StddevUnvoicedSegmentLength']


#### 3. Obtaining X_train & Y_train:

In [108]:
X_train=df_train[df_train.columns.intersection(feature_list)]
X_train.head()

,mfcc4_sma3_amean,F1amplitudeLogRelF0_sma3nz_amean,F1amplitudeLogRelF0_sma3nz_stddevNorm,F2amplitudeLogRelF0_sma3nz_amean,F3amplitudeLogRelF0_sma3nz_amean,alphaRatioV_sma3nz_amean,alphaRatioV_sma3nz_stddevNorm,hammarbergIndexV_sma3nz_amean,slopeUV0-500_sma3nz_amean,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,3.800583,-84.727980,-1.048661,-90.501010,-93.37229,-16.124530,-0.306176,24.537832,-0.036893,0.161118,0.160292
1,-4.607095,-74.447624,-1.202210,-80.129980,-85.28836,-14.999199,-0.435820,26.361210,-0.026273,0.142960,0.153230
2,-3.790331,-83.391560,-1.108741,-87.248660,-91.00723,-15.489925,-0.420070,25.057180,-0.027147,0.157567,0.167011
3,-1.481845,-57.039013,-1.385390,-65.835976,-73.50331,-18.047876,-0.353114,30.874750,-0.008674,0.080040,0.067368
4,-9.107145,-59.857090,-1.386181,-65.951850,-71.99002,-17.023403,-0.442431,27.374958,0.000921,0.097800,0.106605


In [109]:
#replace string values to 0s & 1s in target column
df_train.replace({'class':{"HC":0,"DP":1}},inplace=True)

In [110]:
Y_train=df_train['class']
Y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

#### 4. Resampling the data: 

In [111]:
#we notice an imbalance in the data classes (close to a ratio of 1/3)
val_count=df_train.groupby(['class'])['class'].count()
print(val_count)

class
0    126
1     37
Name: class, dtype: int64


##### 4.1. Upsampling the data

In [112]:
#hence we resample the data to obtain equal number of data points   for each of the two classes:
df_1=df_train[df_train['class']==1]
df_other=df_train[df_train['class']!=1]
df_upsampled=resample(df_1,random_state=42,n_samples=126,replace=True)
df_train_upsampled=pd.concat([df_upsampled,df_other])
df_train_upsampled.reset_index(inplace=True)
#we upsample the minority class to majority class                    he number of data points available for majority class

In [113]:
X_train_resampled=df_train_upsampled[df_train_upsampled.columns.intersection(feature_list)]
Y_train_resampled=df_train_upsampled['class']

##### Robust Feature Selection :

##### computing variance : VARIANCE THRESHOLDING

In [114]:
from sklearn.feature_selection import VarianceThreshold

In [115]:
vt=VarianceThreshold(threshold=1)

In [116]:
df_num=df_train_upsampled.select_dtypes(include='number')
df_num.drop(['class'],axis=1,inplace=True)

In [117]:
df_vnorm=df_num/df_num.mean()

In [118]:
val=df_vnorm.var().sort_values()

In [119]:
val[76]

0.8766758734313158

In [120]:
df_vf=vt.fit(df_num)
mask=vt.get_support()

In [121]:
var_selec=df_num.loc[:,mask]

In [122]:
var_selec.drop(['index'],axis=1,inplace=True)

/tmp/ipykernel_16683/3850877822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_selec.drop(['index'],axis=1,inplace=True)


In [123]:
var_selec.head()

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,loudness_sma3_meanRisingSlope,...,mfcc1V_sma3nz_amean,mfcc2V_sma3nz_amean,mfcc2V_sma3nz_stddevNorm,mfcc3V_sma3nz_amean,mfcc3V_sma3nz_stddevNorm,mfcc4V_sma3nz_amean,mfcc4V_sma3nz_stddevNorm,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,equivalentSoundLevel_dBp
0,23.476286,22.110855,23.837488,25.591496,3.480640,268.02110,366.04483,129.707900,250.56260,1.766769,...,37.463080,14.054592,0.843173,9.901154,1.617111,7.353715,1.588482,-18.105501,28.817120,-44.021973
1,21.260048,20.211780,21.219353,22.181479,1.969698,123.76232,257.21940,17.761978,41.08602,1.348313,...,40.553776,1.769181,6.306235,11.537163,1.268266,4.514918,2.702869,-15.168266,25.892070,-47.172520
2,31.080809,28.899734,30.682478,33.390224,4.490490,295.27902,445.52322,70.410780,142.17552,4.346408,...,34.068380,-7.308782,-2.041490,13.117019,1.359975,-19.985367,-0.796648,-12.688873,23.661478,-34.376457
3,31.807781,30.686207,32.778095,34.603990,3.917782,354.69476,482.07110,134.885120,281.16430,3.355836,...,34.917670,8.852542,1.302755,-3.258625,-6.222845,1.291071,10.276671,-15.309472,25.047730,-36.639270
4,25.438978,23.304884,25.214290,27.379910,4.075026,134.29390,291.90410,53.256367,181.44164,1.789315,...,37.549500,7.677197,1.786598,12.856271,1.268156,5.436445,2.756900,-15.678574,26.755959,-43.762790


##### 4.2. Normalizing the data

In [124]:
#for normalization:
std_Scaler=StandardScaler()
std_Scaler

StandardScaler()

In [125]:
#standard scalar normalization on feature set:
X=std_Scaler.fit_transform(X_train_resampled)

In [126]:
Y=Y_train_resampled

In [127]:
#checking if normalized data has a mean~0 & standard deviation~1:
print(np.mean(X))
print(np.std(X))

-8.394759955318644e-17
1.0


In [128]:
df_train_upsampled['class'].value_counts()

1    126
0    126
Name: class, dtype: int64

### Building the model:

#### 1. Random Forest Classifier:

In [177]:
RF_model=RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=13)

In [130]:
#for unsampled data:
##RF_model.fit(X_train,Y_train)

In [171]:
#for resampled data:
RF_model.fit(X,Y)

RandomForestClassifier(random_state=13)

#### 1.1 Predicting the output on test data:

In [172]:
#obtaining X & Y for test data:
X_test=df_test[df_test.columns.intersection(feature_list)]
df_test.replace({'class':{"HC":0,"DP":1}},inplace=True)
Y_test=df_test['class']

In [173]:
X_test=std_Scaler.transform(X_test)

In [174]:
#predicting on test data:
Y_predict_RF=RF_model.predict(X_test)

#### 1.2 Obtaining a Classification Report:

In [175]:
#viewing accuracy score
print("The RF_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_RF))

The RF_model accuracy is given as :  0.5714285714285714


In [176]:
print(classification_report(Y_test,Y_predict_RF))

              precision    recall  f1-score   support

           0       0.67      0.74      0.71        39
           1       0.23      0.18      0.20        17

    accuracy                           0.57        56
   macro avg       0.45      0.46      0.45        56
weighted avg       0.54      0.57      0.55        56



In [137]:
from sklearn.metrics import classification_report

##### 1.2.1 inference:

* Precision : gives accuracy of positive predictions
* Recall    : gives fraction of correctly identified positive predictions
* F1_score  : gives harmonic mean of precision and recall
* Support   : gives number of occurences of each class in Y_test

--> high precision indicates not many true values were predicted as false.                        
--> high recall indicates most true values were predicted correctly.

#### 2. Decision Tree Classifier:

In [138]:
DF_model=DecisionTreeClassifier(max_depth=7,random_state=42,class_weight='balanced')

In [139]:
DF_model.fit(X,Y)

DecisionTreeClassifier(class_weight='balanced', max_depth=7, random_state=42)

#### 2.1 Predicting the output on test data:

In [140]:
#predicting on test data:
Y_predict_DF=DF_model.predict(X_test)

In [141]:
#viewing accuracy score:
print("The DF_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_DF))

The DF_model accuracy is given as :  0.625


#### 2.2 Obtaining a Classification Report:

In [142]:
print(classification_report(Y_test,Y_predict_DF))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71        39
           1       0.41      0.53      0.46        17

    accuracy                           0.62        56
   macro avg       0.59      0.60      0.59        56
weighted avg       0.66      0.62      0.64        56



#### 3. Logistic Regression Model:

In [143]:
LR_model=LogisticRegression(class_weight='balanced')

In [144]:
LR_model.fit(X,Y)

LogisticRegression(class_weight='balanced')

#### 3.1 Predicting the output on test data:

In [145]:
#predicting on test data:
Y_predict_LR=LR_model.predict(X_test)

In [146]:
#viewing accuracy score:
print("The LR_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_LR))

The LR_model accuracy is given as :  0.4642857142857143


#### 3.2 Obtaining a Classification Report:

In [147]:
print(classification_report(Y_test,Y_predict_LR))

              precision    recall  f1-score   support

           0       0.71      0.38      0.50        39
           1       0.31      0.65      0.42        17

    accuracy                           0.46        56
   macro avg       0.51      0.52      0.46        56
weighted avg       0.59      0.46      0.48        56



#### 4. Support Vector Machine Model:

In [165]:
SVM_model = SVC(kernel='rbf',C=10, gamma=1, class_weight='balanced')
#rbf: gaussian radial basis function

In [166]:
SVM_model.fit(X,Y)

SVC(C=10, class_weight='balanced', gamma=1)

#### 4.1 Predicting the output on test data:

In [167]:
#predicting on test data:
Y_predict_SVM=SVM_model.predict(X_test)

In [168]:
#viewing accuracy score:
print("The SVM_accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_SVM))

The SVM_accuracy is given as :  0.6785714285714286


#### 4.2 Obtaining a Classification Report:

In [169]:
print(classification_report(Y_test,Y_predict_SVM))

              precision    recall  f1-score   support

           0       0.70      0.95      0.80        39
           1       0.33      0.06      0.10        17

    accuracy                           0.68        56
   macro avg       0.52      0.50      0.45        56
weighted avg       0.59      0.68      0.59        56



<font color=red>**C=10 & gamma=1 with resampled gives the best results**</font>

#### 4.3 Hyperparameter Tuning (by Grid Search):

In [153]:
from sklearn.model_selection import GridSearchCV

#### 4.3.1 Building the model:

In [154]:
param_grid= {'C':[0.1,1,10,100,1000],
             'gamma':[1,0.1,0.01,0.001,0.0001],
             'kernel':['rbf']
            }

In [155]:
grid= GridSearchCV(SVC(),param_grid=param_grid,refit=True,verbose=3)

In [156]:
#grid.fit(Xp,Y)
grid.fit(X,Y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.880 total time=   0.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.700 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.840 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.745 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.471 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.820 total time=   0.1s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.620 total time=   0.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.490 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.843 total time=   0.0s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.840 total time=   0.0s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.960 total time=   0.0s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 1/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.667 total time=   0.1s
[CV 2/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.804 total time=   0.0s
[CV 3/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.840 total time=   0.1s
[CV 4/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.920 total time=   0.1s
[CV 5/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.700 total time=   0.0s
[CV 1/5] END ...C=1000, gamma=0.001, kernel=rbf;, score=0.471 total time=   0.0s
[CV 2/5] END ...C=1000, gamma=0.001, kernel=rbf;, score=0.569 total time=   0.0s
[CV 3/5] END ...C=1000, gamma=0.001, kernel=rbf;, score=0.660 total time=   0.0s
[CV 4/5] END ...C=1000, gamm

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [157]:
#finding the best parameters:
print(grid.best_params_)

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}


In [158]:
##array of selected features

#selected_features=np.array(req_features['Unnamed: 0'])
#print(selected_features)

#feat_imp=permutation_importance(SVM_model,X_test,Y_test)

#sorted_importance=feat_imp.importances_mean.argsort()
#ax=plt.barh(selected_features[sorted_importance],feat_imp.importances_mean[sorted_importance])
#plt.xlabel("Feature Importance")

#def feat_imp_selected():
    #s=feat_imp.importances_mean[sorted_importance]
    #imp=selected_features[sorted_importance]
    #i=0
    #y=[]
    #for i in range (0,11):
        #if s[i] <-0.012 or s[i]>0.005:
            #y.append(imp[i])
    #return y

#imp_features_selected=feat_imp_selected()
#print(type(imp_features_selected))
#print(imp_features_selected)

#s=feat_imp.importances_mean[sorted_importance]
#print(len(s))
#print(len(imp_features_selected))

## Building a model without feature selection:

In [ ]:
#X1=df_train.drop(['file','start','end','class'],axis=1)

#### Normalizing the features:

In [ ]:
#X1=std_Scaler.fit_transform(X1)

In [ ]:
#Y1=Y_train

#### Model with imbalanced data:

In [ ]:
#SVM_model1= SVC(kernel='rbf',C=0.1, gamma=0.01, class_weight='balanced')

In [ ]:
#SVM_model1.fit(X1,Y1)

In [ ]:
#predicting on test data:
#Y_predict_SVM1=SVM_model1.predict(X1_test)

In [ ]:
#viewing accuracy score:
#print("The optimised SVM_accuracy without feature selection is given as : ",metrics.accuracy_score(Y1_test,Y_predict_SVM1))

In [ ]:
#print(classification_report(Y1_test,Y_predict_SVM1))

#### Model with resampled data:

In [ ]:
#resampled df:
df_train_upsampled.shape

In [178]:
X1=df_train_upsampled.drop(['index','file','start','end','class'],axis=1)
Y1=df_train_upsampled['class']

In [179]:
#normalizing the features:
X1=std_Scaler.fit_transform(X1)

In [180]:
#obtaining X & Y for test data:
X1_test=df_test.drop(['file','start','end','class'],axis=1)
df_test.replace({'class':{"HC":0,"DP":1}},inplace=True)

In [181]:
X1_test=std_Scaler.transform(X1_test)
Y1_test=Y_test

In [182]:
grid1= GridSearchCV(SVC(),param_grid=param_grid,refit=True,verbose=3)

In [183]:
grid1.fit(X1,Y1)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.700 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.780 total time=   0.1s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.490 total time=   0.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.490 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.680 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.760 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.490 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.980 total time=   0.0s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.961 total time=   0.0s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.960 total time=   0.1s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=1.000 total time=   0.1s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.900 total time=   0.0s
[CV 1/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.882 total time=   0.0s
[CV 2/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.843 total time=   0.0s
[CV 3/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.880 total time=   0.0s
[CV 4/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.940 total time=   0.0s
[CV 5/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.800 total time=   0.0s
[CV 1/5] END ...C=1000, gamma=0.001, kernel=rbf;, score=0.784 total time=   0.1s
[CV 2/5] END ...C=1000, gamma=0.001, kernel=rbf;, score=0.843 total time=   0.1s
[CV 3/5] END ...C=1000, gamm

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [187]:
print(grid1.best_params_)

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [192]:
SVM_model1= SVC(kernel='rbf',C=10, gamma=0.01, class_weight='balanced')
SVM_model1.fit(X1,Y1)

SVC(C=10, class_weight='balanced', gamma=0.01)

In [186]:
#not normalized
#svmmodeln=SVC(kernel='rbf',C=10, gamma=0.01, class_weight='balanced')
#svmmodeln.fit(X_1,Y1)

In [193]:
Y_predict_SVM_all_features=SVM_model1.predict(X1_test)

In [194]:
#viewing accuracy score
print("The SVM_accuracy for resampled data is given as : ",metrics.accuracy_score(Y1_test,Y_predict_SVM_all_features))

The SVM_accuracy for resampled data is given as :  0.6785714285714286


In [195]:
print(classification_report(Y1_test,Y_predict_SVM_all_features))

              precision    recall  f1-score   support

           0       0.72      0.87      0.79        39
           1       0.44      0.24      0.31        17

    accuracy                           0.68        56
   macro avg       0.58      0.55      0.55        56
weighted avg       0.64      0.68      0.64        56



### Understanding Feature Importance for all features:

#### Obtaining selected features from best performing classifier model:

##### with normalization:

In [ ]:
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt

In [ ]:
r=df_train_upsampled.drop(['index','file','start','end','class'],axis=1)
feature_names=r.columns.tolist()
#featureseries=pd.Series(feature_names)
features=np.array(feature_names)

In [ ]:
type(r)

In [ ]:
feat_imp_all=permutation_importance(SVM_model1,X1_test,Y1_test)
sorted_importance_all=feat_imp_all.importances_mean.argsort()
f=features[sorted_importance_all]
v=feat_imp_all.importances_mean[sorted_importance_all]
plt.barh(f,v)
plt.xlabel("Feature Importance for all Features")

In [ ]:
def feat_imp_all():
    i=0
    x=[]
    for i in range (0,88):
        if v[i] <-0.015 or v[i]>0.01:
            x.append(f[i])
    return x

In [ ]:
imp_features_all=feat_imp_all()
print(type(imp_features_all))
print(imp_features_all)

In [ ]:
print(len(f))
print(len(imp_features_all))

##### without normalization:

In [ ]:
#feat_imp_all_n=permutation_importance(svmmodeln,X1__test,Y1_test)
#sorted_importance_all_n=feat_imp_all_n.importances_mean.argsort()
#a=features[sorted_importance_all_n]
#b=feat_imp_all_n.importances_mean[sorted_importance_all_n]
#plt.barh(a,b)
#plt.xlabel("Feature Importance for all Features without Norm.")

### Finding overlap between selected features from (1) statistical analysis & (2) classifier model :

In [ ]:
def overlapping_features(l1,l2):
    l3=[value for value in l1 if value in l2]
    return l3

In [ ]:
f=overlapping_features(feature_list,imp_features_all)

In [ ]:
f


### Feature Combination:

#### PCA (on selected features):

In [196]:
from sklearn.decomposition import PCA

#### reducing to a single dimension:

In [ ]:
pca=PCA(n_components=1)

In [ ]:
x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

In [ ]:
#convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1"])
pca_df['y']=Y
pca_df.tail()

In [ ]:
#evr provides the amount of info or variance each principal component holds after projecting the data to a lower dimensional space:
print("evr is {}".format(pca.explained_variance_ratio_))

__inference__: 
* principal component 1 holds 52.9%

#### reducing to 2 dimensions:

In [ ]:
pca=PCA(n_components=2)

In [ ]:
x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

In [ ]:
##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2"])
#pca_df['y']=Y
pca_df.head()

In [ ]:
new_df=pd.DataFrame(x_trainpca)
y=Y.tolist()
new_df['y']=y
new_df.tail(3)
#type(new_df)

In [ ]:
#evr provides the amount of info or variance each principal component holds after projecting the data to a lower dimensional space:
print("evr is {}".format(pca.explained_variance_ratio_))

__inference__: 
* principal component 1 holds 52.9%
* principal component 2 holds 24.3%

In [ ]:
ClassAIndices=np.where(Y==0) #this gives an array
ClassAIndices=ClassAIndices[0].tolist()
ClassBIndices=np.where(Y==1)
ClassBIndices=ClassBIndices[0].tolist()

In [ ]:
x_HC=x_trainpca[ClassAIndices,:]
x_DP=x_trainpca[ClassBIndices,:]
y_HC=Y[ClassAIndices]
y_DP=Y[ClassBIndices]

In [ ]:
plt.scatter(x_HC[:,0],x_HC[:,1], color='green',s=50, marker='o',alpha=0.9,edgecolor='k',label='HC')
plt.scatter(x_DP[:,0],x_DP[:,1], color='red',s=50, marker='o',alpha=0.9,edgecolor='k',label='DP') 
plt.xlabel('Principal Component 1') 
plt.ylabel('Principal Component 2') 
plt.legend()
plt.show()

#### reducing to 4 dimensions:

In [197]:
pca=PCA(n_components=4)

In [198]:
x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

In [199]:
##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3","PCA4"])
pca_df.head()

,PCA1,PCA2,PCA3,PCA4
0,4.348515,-1.146364,0.376333,-0.276584
1,1.560722,-0.170687,-0.249257,0.988435
2,-5.282826,0.742804,-0.319651,-0.231372
3,-0.103772,-0.603448,-0.051544,0.155027
4,0.215474,-0.824120,-0.709310,1.020471


In [162]:
#evr provides the amount of info or variance each principal component holds after projecting the data to a lower dimensional space:
print("evr is {}".format(pca.explained_variance_ratio_))

evr is [0.5290246  0.24266165 0.09908918 0.05339629]


In [227]:
df_train_upsampled

0      /home/biswajit/Documents/jupyter_scribe/AVEC_2...
1      /home/biswajit/Documents/jupyter_scribe/AVEC_2...
2      /home/biswajit/Documents/jupyter_scribe/AVEC_2...
3      /home/biswajit/Documents/jupyter_scribe/AVEC_2...
4      /home/biswajit/Documents/jupyter_scribe/AVEC_2...
                             ...                        
247    /home/biswajit/Documents/jupyter_scribe/AVEC_2...
248    /home/biswajit/Documents/jupyter_scribe/AVEC_2...
249    /home/biswajit/Documents/jupyter_scribe/AVEC_2...
250    /home/biswajit/Documents/jupyter_scribe/AVEC_2...
251    /home/biswajit/Documents/jupyter_scribe/AVEC_2...
Name: file, Length: 252, dtype: object

In [272]:
##convert principal components for each sample from numpy array to df:
pca_dft=pd.DataFrame(x_testpca,columns=["PCA1","PCA2","PCA3","PCA4"])
pca_dft["class"]=Y_test
pca_dft["speaker"]=spt
pca_dft.tail()

,PCA1,PCA2,PCA3,PCA4,class,speaker
51,-0.645317,-1.425304,-0.255952,0.251877,0,712
52,0.601740,1.690538,0.880718,0.886949,0,715
53,2.375223,-0.016901,-0.607982,0.002257,1,716
54,-0.627001,-0.058607,0.066025,0.235613,0,717
55,-1.984048,3.999375,1.254256,1.193432,0,718


In [273]:
pca_dft.to_csv("pcatest_best.csv")

In [252]:
##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3","PCA4"])
pca_df["class"]=Y
pca_df["speaker"]=sp
pca_df.tail()

,PCA1,PCA2,PCA3,PCA4,class,speaker
247,2.668233,-0.446498,0.180630,0.098473,0,695
248,2.448544,-1.648400,-0.333052,-0.113243,0,697
249,0.602317,2.003996,-2.570648,0.701187,0,702
250,5.416251,0.743620,-1.263139,1.409929,0,703
251,0.859193,-0.472979,-1.578957,0.721803,0,707


In [253]:
pca_df.to_csv("pca_best.csv")

In [163]:
ClassAIndices=np.where(Y==0) #this gives an array
ClassAIndices=ClassAIndices[0].tolist()
ClassBIndices=np.where(Y==1)
ClassBIndices=ClassBIndices[0].tolist()

In [ ]:
x_HC=x_trainpca[ClassAIndices,:]
x_DP=x_trainpca[ClassBIndices,:]
y_HC=Y[ClassAIndices]
y_DP=Y[ClassBIndices]

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
plt.rcParams["figure.figsize"]=[5,5]
plt.rcParams["figure.autolayout"]=True
fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
ax.scatter3D(x_HC[:,0],x_HC[:,1],x_HC[:,2], color='green',s=50, marker='o',alpha=0.9,edgecolor='k',label='HC')
ax.scatter3D(x_DP[:,0],x_DP[:,1],x_DP[:,2], color='red',s=50, marker='o',alpha=0.9,edgecolor='k',label='DP') 
ax.set_xlabel('Principal Component 1') 
ax.set_ylabel('Principal Component 2') 
ax.set_zlabel('Principal Component 3')
plt.legend()
plt.show()

In [ ]:
plt.rcParams["figure.figsize"]=[5,5]
plt.rcParams["figure.autolayout"]=True
fig=plt.figure()
ax=fig.add_subplot(projection='3d')

ax.scatter3D(x_HC[:,0],x_HC[:,1],x_HC[:,2], color='green',s=50, marker='o',alpha=0.9,edgecolor='k',label='HC')
ax.scatter3D(x_DP[:,0],x_DP[:,1],x_DP[:,2], color='red',s=50, marker='o',alpha=0.9,edgecolor='k',label='DP') 
ax.set_xlabel('Principal Component 1') 
ax.set_ylabel('Principal Component 2') 
ax.set_zlabel('Principal Component 3')
plt.legend()
for angle in range(0, 360*4 + 1):
    # Normalize the angle to the range [-180, 180] for display
    angle_norm = (angle + 180) % 360 - 180

    # Cycle through a full rotation of elevation, then azimuth, roll, and all
    elev = azim = roll = 0
    if angle <= 360:
        elev = angle_norm
    elif angle <= 360*2:
        azim = angle_norm
    elif angle <= 360*3:
        roll = angle_norm
    else:
        elev = azim = roll = angle_norm

    # Update the axis view and title
    ax.view_init(elev, azim, roll)
    plt.draw()
    plt.pause(0.001)
#plt.show()

#### reducing to 4 dimensions:

In [ ]:
pca=PCA(n_components=3)

In [ ]:
Xp_test=df_test[df_test.columns.intersection(f)]
Xp_test=std_Scaler.transform(Xp_test)

In [ ]:
Xp=df_train_upsampled[df_train_upsampled.columns.intersection(f)]
Xp=std_Scaler.fit_transform(Xp)

In [ ]:
x_trainpca=pca.fit_transform(Xp)
x_testpca=pca.transform(Xp_test)

In [ ]:
##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3"])
pca_df["y"]=Y
pca_df.tail()

In [ ]:
#evr provides the amount of info or variance each principal component holds after projecting the data to a lower dimensional space:
print("evr is {}".format(pca.explained_variance_ratio_))

__inference__: 
* principal component 1 holds 52.9%
* principal component 2 holds 24.3%
* principal component 3 holds 9.91%
* principal component 4 holds 5.33% </br>
__adding them all up we get over 92.4%(>90). Hence  we decide to take 4 components.__

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(pca_df,hue="y")

##### RF model:

In [206]:
RF_model.fit(x_trainpca,Y)

RandomForestClassifier(class_weight='balanced', random_state=13)

In [207]:
Y_predict_RFpca=RF_model.predict(x_testpca)

In [208]:
#viewing accuracy score
print("The RF_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_RFpca))

The RF_model accuracy is given as :  0.7142857142857143


In [209]:
print(classification_report(Y_test,Y_predict_RFpca))

              precision    recall  f1-score   support

           0       0.74      0.90      0.81        39
           1       0.56      0.29      0.38        17

    accuracy                           0.71        56
   macro avg       0.65      0.60      0.60        56
weighted avg       0.69      0.71      0.68        56



##### DF model:

In [210]:
DF_model.fit(x_trainpca,Y)

DecisionTreeClassifier(class_weight='balanced', max_depth=7, random_state=42)

In [211]:
Y_predict_DFpca=DF_model.predict(x_testpca)

In [212]:
print("The DF_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_DFpca))

The DF_model accuracy is given as :  0.6607142857142857
The DF_model accuracy is given as :  0.6607142857142857


In [213]:
print(classification_report(Y_test,Y_predict_DFpca))

              precision    recall  f1-score   support

           0       0.83      0.64      0.72        39
           1       0.46      0.71      0.56        17

    accuracy                           0.66        56
   macro avg       0.65      0.67      0.64        56
weighted avg       0.72      0.66      0.67        56



##### LR model:

In [214]:
LR_model.fit(x_trainpca,Y)

LogisticRegression(class_weight='balanced')

In [215]:
Y_predict_LRpca=LR_model.predict(x_testpca)

In [216]:
print("The LR_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_LRpca))

The LR_model accuracy is given as :  0.39285714285714285


In [217]:
print(classification_report(Y_test,Y_predict_LRpca))

              precision    recall  f1-score   support

           0       0.65      0.28      0.39        39
           1       0.28      0.65      0.39        17

    accuracy                           0.39        56
   macro avg       0.46      0.46      0.39        56
weighted avg       0.54      0.39      0.39        56



##### SVM model:

In [218]:
grid.fit(x_trainpca,Y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.490 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.820 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.760 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.725 total time=   0.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.471 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.660 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.740 total time=   0.1s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.620 total time=   0.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.490 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

[CV 2/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.863 total time=   0.1s
[CV 3/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.880 total time=   0.0s
[CV 4/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.960 total time=   0.1s
[CV 5/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.780 total time=   0.1s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.804 total time=   0.1s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.765 total time=   0.1s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.820 total time=   0.1s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.900 total time=   0.2s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.740 total time=   0.1s
[CV 1/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.588 total time=   0.2s
[CV 2/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.549 total time=   0.1s
[CV 3/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.620 total time=   0.2s
[CV 4/5] END ....C=1000, gam

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [219]:
#finding the best parameters:
print(grid.best_params_)

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}


In [277]:
SVM_modelpca= SVC(kernel='rbf',C=10, gamma=1, class_weight='balanced',probability=True)
SVM_modelpca.fit(x_trainpca,Y)

SVC(C=10, class_weight='balanced', gamma=1, probability=True)

In [284]:
Y_predict_SVMpca=SVM_modelpca.predict(x_testpca)
Y_predict_SVMpca

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [291]:
Y.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [292]:
prob_tr=SVM_modelpca.predict_proba(x_trainpca)
prob_tr

array([[0.12895453, 0.87104547],
       [0.12903685, 0.87096315],
       [0.12905769, 0.87094231],
       [0.12899098, 0.87100902],
       [0.12897579, 0.87102421],
       [0.12898336, 0.87101664],
       [0.12899543, 0.87100457],
       [0.12899543, 0.87100457],
       [0.12900759, 0.87099241],
       [0.12899541, 0.87100459],
       [0.12900759, 0.87099241],
       [0.12899529, 0.87100471],
       [0.12891722, 0.87108278],
       [0.12896681, 0.87103319],
       [0.12900759, 0.87099241],
       [0.12890555, 0.87109445],
       [0.12896681, 0.87103319],
       [0.12899098, 0.87100902],
       [0.12897324, 0.87102676],
       [0.1290953 , 0.8709047 ],
       [0.12891722, 0.87108278],
       [0.12905367, 0.87094633],
       [0.12910039, 0.87089961],
       [0.12902758, 0.87097242],
       [0.12897583, 0.87102417],
       [0.12903685, 0.87096315],
       [0.12899529, 0.87100471],
       [0.12895965, 0.87104035],
       [0.1291253 , 0.8708747 ],
       [0.12899098, 0.87100902],
       [0.

In [298]:
svm_prob_tr=pd.DataFrame(prob_tr,columns=['prob_hc','prob_dp'])
svm_prob_tr["speaker"]=sp
svm_prob_tr["class"]=Y
svm_prob_tr.to_csv("svm_prob_best.csv")

In [299]:
prob=SVM_modelpca.predict_proba(x_testpca)

In [300]:
svm_prob=pd.DataFrame(prob,columns=['prob_hc','prob_dp'])
svm_prob["speaker"]=spt
svm_prob["class"]=Y_test
svm_prob.to_csv("svm_prob_tr_best.csv")

In [286]:
svm_prob

,prob_hc,prob_dp,speaker,class
0,0.912154,0.087846,600,0
1,0.947746,0.052254,602,1
2,0.536509,0.463491,604,1
3,0.952286,0.047714,605,0
4,0.230067,0.769933,606,0
5,0.858174,0.141826,607,0
6,0.713742,0.286258,609,0
7,0.987866,0.012134,615,0
8,0.847955,0.152045,618,0
9,0.989703,0.010297,619,0


In [276]:
#viewing accuracy score
print("The SVM_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_SVMpca))

The SVM_model accuracy is given as :  0.75


In [70]:
print(classification_report(Y_test,Y_predict_SVMpca))

              precision    recall  f1-score   support

           0       0.78      0.90      0.83        39
           1       0.64      0.41      0.50        17

    accuracy                           0.75        56
   macro avg       0.71      0.65      0.67        56
weighted avg       0.73      0.75      0.73        56



#### tSNE (on selected features):

In [ ]:
from sklearn.manifold import TSNE
import time
time_start=time.time()
tsne=TSNE(n_components=2,verbose=1,perplexity=40,n_iter=300)

In [ ]:
xtrain_tsne=tsne.fit_transform(X)

In [ ]:
x_testtsne=tsne.transform(X_test)

In [ ]:
xt_HC=xtrain_tsne[ClassAIndices,:]
xt_DP=xtrain_tsne[ClassBIndices,:]
yt_HC=Y[ClassAIndices]
yt_DP=Y[ClassBIndices]

In [ ]:
plt.scatter(xt_HC[:,0],xt_HC[:,1], color='green',s=50, marker='o',alpha=0.9,edgecolor='k',label='HC')
plt.scatter(xt_DP[:,0],xt_DP[:,1], color='red',s=50, marker='o',alpha=0.9,edgecolor='k',label='DP') 
plt.xlabel('tSNE Component 1') 
plt.ylabel('tSNE Component 2') 
plt.legend()
plt.show()

#### LDA (on selected features):

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
lda=LDA(n_components=1)

In [ ]:
xtrain_lda=lda.fit_transform(X,Y)

In [ ]:
x_testlda=lda.transform(X_test)

In [ ]:
SVM_modellda= SVC(kernel='rbf',C=10, gamma=0.01, class_weight='balanced')
SVM_modellda.fit(xtrain_lda,Y)

In [ ]:
SVM_modellda.fit(xtrain_lda,Y)
print(optSVM_model.best_params_)

In [ ]:
Y_predict_SVMlda=SVM_modellda.predict(x_testlda)
print("The SVM(LDA)_model accuracy is given as : ",metrics.accuracy_score(Y_test,Y_predict_SVMlda))

In [ ]:
print(classification_report(Y_test,Y_predict_SVMlda))